# Introduction
Rarely do we find ourselves demanding to know how predictive models do trivial things like determining the latest and greatest cat clip to put at the top of a Tik Tok feed. But for high stakes predictions or when conducting machine learning research, models cannot be treated as black boxes. In order to understand how input features affect model predictions, SHAP (SHapley Additive exPlanations), using theory by American mathematician Lloyd Shapley, make ML models interpretable.

We will use some data on the proportion of salmon of different ages from NOAA to train a model that predicts the the amount of salmon in the run

# Create Model   

In [1]:
import lightgbm as lgb


X_train = pd.read_parquet(path.join(file_dir), "X_train.parquet")
y_train = pd.read_parquet(path.join(file_dir), "X_test.parquet")
X_test = pd.read_parquet(path.join(file_dir), "y_train.parquet")
y_test = pd.read_parquet(path.join(file_dir), "y_test.parquet")

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

OSError: dlopen(/Users/aholten/holten/pydata-seattle-fugue/.env/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/aholten/holten/pydata-seattle-fugue/.env/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

# Calculate SHAP